In [1]:
import pandas as pd

In [2]:
path_cursos = 'data/CADASTRO_CURSOS_2020.CSV'
path_ies = 'data/CADASTRO_IES_2020.CSV'
path_dicio = 'data/dicionrio_dados_educao_superior.xlsx'

In [3]:
df_cursos = pd.read_csv(path_cursos, sep=';', encoding_errors='replace', low_memory=False)
df_ies = pd.read_csv(path_ies, sep=';', encoding_errors='replace')
df_dicio = pd.read_excel(path_dicio)

In [4]:
df_cursos

,NU_ANO_CENSO,NO_REGIAO,CO_REGIAO,NO_UF,SG_UF,CO_UF,NO_MUNICIPIO,CO_MUNICIPIO,IN_CAPITAL,TP_ORGANIZACAO_ACADEMICA,...,MAT_APOIO_SOCIAL,CONC_APOIO_SOCIAL,ATIV_EXTRACURRICULAR,ING_ATIV_EXTRACURRICULAR,MAT_ATIV_EXTRACURRICULAR,CONC_ATIV_EXTRACURRICULAR,MOB_ACADEMICA,ING_MOB_ACADEMICA,MAT_MOB_ACADEMICA,CONC_MOB_ACADEMICA
0,2020,Norte,1,Rondônia,RO,11,Alta Floresta D'Oeste,1100015.0,0.0,1,...,0,0,0,0,0,0,0,0,0,0
1,2020,Norte,1,Rondônia,RO,11,Alta Floresta D'Oeste,1100015.0,0.0,1,...,0,0,0,0,0,0,0,0,0,0
2,2020,Norte,1,Rondônia,RO,11,Alta Floresta D'Oeste,1100015.0,0.0,1,...,0,0,0,0,0,0,0,0,0,0
3,2020,Norte,1,Rondônia,RO,11,Alta Floresta D'Oeste,1100015.0,0.0,1,...,0,0,0,0,0,0,0,0,0,0
4,2020,Norte,1,Rondônia,RO,11,Alta Floresta D'Oeste,1100015.0,0.0,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
335624,2020,Exterior,7,Exterior,NaN,71,Exterior,NaN,NaN,2,...,0,0,0,0,0,0,0,0,0,0
335625,2020,Exterior,7,Exterior,NaN,71,Exterior,NaN,NaN,2,...,0,0,0,0,0,0,0,0,0,0
335626,2020,Exterior,7,Exterior,NaN,71,Exterior,NaN,NaN,2,...,0,0,0,0,0,0,0,0,0,0
335627,2020,Exterior,7,Exterior,NaN,71,Exterior,NaN,NaN,2,...,0,0,0,0,0,0,0,0,0,0


In [5]:
df_ies

,NU_ANO_CENSO,NO_REGIAO_IES,CO_REGIAO_IES,NO_UF_IES,SG_UF_IES,CO_UF_IES,NO_MUNICIPIO_IES,CO_MUNICIPIO_IES,IN_CAPITAL_IES,NO_MESORREGIAO_IES,...,DOC_EX_60_MAIS,DOC_EX_BRANCA,DOC_EX_PRETA,DOC_EX_PARDA,DOC_EX_AMARELA,DOC_EX_IND�GENA,DOC_EX_COR_ND,DOC_EX_BRA,DOC_EX_EST,DOC_EX_COM_DEFICI�NCIA
0,2020,Norte,1,Rond�nia,RO,11,Ariquemes,1100023,0,Leste Rondoniense,...,2,18,1,8,0,0,27,54,0,0
1,2020,Norte,1,Rond�nia,RO,11,Ariquemes,1100023,0,Leste Rondoniense,...,3,45,5,17,2,0,0,67,2,0
2,2020,Norte,1,Rond�nia,RO,11,Cacoal,1100049,0,Leste Rondoniense,...,9,114,2,31,2,0,3,152,0,0
3,2020,Norte,1,Rond�nia,RO,11,Cacoal,1100049,0,Leste Rondoniense,...,1,30,1,4,0,0,4,39,0,0
4,2020,Norte,1,Rond�nia,RO,11,Cacoal,1100049,0,Leste Rondoniense,...,0,14,0,7,0,0,18,39,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2452,2020,Centro-Oeste,5,Distrito Federal,DF,53,Bras�lia,5300108,1,Distrito Federal,...,0,14,0,3,0,0,0,17,0,0
2453,2020,Centro-Oeste,5,Distrito Federal,DF,53,Bras�lia,5300108,1,Distrito Federal,...,0,3,0,1,0,0,0,4,0,0
2454,2020,Centro-Oeste,5,Distrito Federal,DF,53,Bras�lia,5300108,1,Distrito Federal,...,2,5,0,1,0,0,5,11,0,0
2455,2020,Centro-Oeste,5,Distrito Federal,DF,53,Bras�lia,5300108,1,Distrito Federal,...,0,6,0,4,0,0,2,12,0,0


In [6]:
df_dicio

,Unnamed: 0,Alteração de nomenclatura,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19
0,NaN,Variável nova,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,Descontinuidade,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Cadastro_IES,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86,77,DOC_EX_INDÍGENA,Quantidade de docentes em exercício - Cor/Raç...,Num,8,NaN,NaN,s,s,s,s,s,s,s,s,s,s,s,s,NaN
87,78,DOC_EX_COR_ND,Quantidade de docentes em exercício - Cor/Raç...,Num,8,NaN,NaN,s,s,s,s,s,s,s,s,s,s,s,s,NaN
88,79,DOC_EX_BRA,Quantidade de docente em exercício - nacionali...,Num,8,NaN,NaN,s,s,s,s,s,s,s,s,s,s,s,s,NaN
89,80,DOC_EX_EST,Quantidade de docente em exercício - nacionali...,Num,8,NaN,NaN,s,s,s,s,s,s,s,s,s,s,s,s,NaN
